In [ ]:
for d in datasets_list:
    if "wikimedia" in d:
        print(d)
        break

In [ ]:
import yaml

with open("../data/huggingface-data-README.yml", "r") as stream:
    try:
        dataset_readme = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


In [ ]:
downloads= set(['en', 'fr'])
configs = dataset_readme['configs']
conf_list = []
for conf in configs:
    if conf['config_name'].split('.')[-1] in downloads:
        print(conf['config_name'])
        cnf = conf['config_name']
        conf_list.append(cnf)

In [ ]:
import re

def get_parquet_file_list(text:str):
    pattern = r'download href="/datasets/wikimedia/wikipedia/resolve/main/.*\.parquet\?download=true">\d* MB'
    substrings = re.findall(pattern, text)
    return substrings


In [ ]:
import requests

def get_one_parqet_file_per_conf(cnf):
    base_url = 'https://huggingface.co/datasets/wikimedia/wikipedia/tree/main/'
    url = base_url  + cnf
    resp = requests.get(url)
    text = resp.content.decode('utf-8')
    parquet_files = get_parquet_file_list(text)
    
    urls_parquet = []
    for f in parquet_files:
        f = f.replace('download href="', '').replace("MB", '')
    
        url_p = f.split('">')[0]
        size_p = f.split('">')[-1]
        urls_parquet.append([url_p.strip(), size_p.strip()])
    
    
    urls_parquet = sorted(urls_parquet, key=lambda u: u[-1])

    min_size = 150 # MB, try to get the small file, but greater than this if possible
    if len(urls_parquet) ==1: 
        return urls_parquet[0]
    else:
        result = None
        for u in urls_parquet:
            result = u
            if int(result[-1]) > min_size:
                return result
        return result
            
    

In [ ]:
from datasets import load_dataset

# ds = load_dataset("wikimedia/wikipedia", "20231101.en", data_files = ['train-00000-of-00041.parquet'])

In [ ]:

parquet_file_paths = []
for conf in conf_list:
     parquet_file_paths.append((get_one_parqet_file_per_conf(conf), conf.split('.')[-1]))

parquet_file_paths  

In [ ]:
import os

def download_file(path, local_filename):
    base_url = "https://huggingface.co"
    url = base_url + path
    response = requests.get(url, stream=True)
    
    with open(local_filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)


for pths in parquet_file_paths:
    file_path  = pths[0][0]
    lang = pths[1]
    
    # print(file_path)
    file_save_as = '../data/parquet/' + lang+"."+file_path.split('?')[0].split('/')[-1]
    
    # print(file_save_as)

    if not os.path.exists(file_save_as):
        print(f'downloading {file_path} to {file_save_as}')
        download_file(file_path, file_save_as)
    else: 
        print(f'{file_save_as} is existing.')



In [ ]:
parquet_file_paths

In [ ]:
import pandas as pd
df = pd.read_parquet(file_save_as, engine='fastparquet')
text = " ".join(df["text"].values.tolist())

print(text[:100])


In [ ]:
len(text)

In [14]:
for d in datasets_list:
    if "wikimedia" in d:
        print(d)
        break

wikimedia/wikipedia


In [21]:
import yaml

with open("../data/huggingface-data-README.yml", "r") as stream:
    try:
        dataset_readme = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)


In [72]:
downloads= set(['en', 'fr'])
configs = dataset_readme['configs']
conf_list = []
for conf in configs:
    if conf['config_name'].split('.')[-1] in downloads:
        print(conf['config_name'])
        cnf = conf['config_name']
        conf_list.append(cnf)

20231101.en
20231101.fr


In [82]:
import re

def get_parquet_file_list(text:str):
    pattern = r'download href="/datasets/wikimedia/wikipedia/resolve/main/.*\.parquet\?download=true">\d* MB'
    substrings = re.findall(pattern, text)
    return substrings


In [83]:
import requests

def get_one_parqet_file_per_conf(cnf):
    base_url = 'https://huggingface.co/datasets/wikimedia/wikipedia/tree/main/'
    url = base_url  + cnf
    resp = requests.get(url)
    text = resp.content.decode('utf-8')
    parquet_files = get_parquet_file_list(text)
    
    urls_parquet = []
    for f in parquet_files:
        f = f.replace('download href="', '').replace("MB", '')
    
        url_p = f.split('">')[0]
        size_p = f.split('">')[-1]
        urls_parquet.append([url_p.strip(), size_p.strip()])
    
    
    urls_parquet = sorted(urls_parquet, key=lambda u: u[-1])

    min_size = 150 # MB, try to get the small file, but greater than this if possible
    if len(urls_parquet) ==1: 
        return urls_parquet[0]
    else:
        result = None
        for u in urls_parquet:
            result = u
            if int(result[-1]) > min_size:
                return result
        return result
            
    

In [84]:
from datasets import load_dataset

# ds = load_dataset("wikimedia/wikipedia", "20231101.en", data_files = ['train-00000-of-00041.parquet'])

In [87]:

parquet_file_paths = []
for conf in conf_list:
     parquet_file_paths.append((get_one_parqet_file_per_conf(conf), conf.split('.')[-1]))

parquet_file_paths  

[(['/datasets/wikimedia/wikipedia/resolve/main/20231101.en/train-00028-of-00041.parquet?download=true',
   '188'],
  'en'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.fr/train-00011-of-00017.parquet?download=true',
   '169'],
  'fr')]

In [96]:
import os

def download_file(path, local_filename):
    base_url = "https://huggingface.co"
    url = base_url + path
    response = requests.get(url, stream=True)
    
    with open(local_filename, 'wb') as file:
        for chunk in response.iter_content(chunk_size=128):
            file.write(chunk)


for pths in parquet_file_paths:
    file_path  = pths[0][0]
    lang = pths[1]
    
    # print(file_path)
    file_save_as = '../data/parquet/' + lang+"."+file_path.split('?')[0].split('/')[-1]
    
    # print(file_save_as)

    if not os.path.exists(file_save_as):
        print(f'downloading {file_path} to {file_save_as}')
        download_file(file_path, file_save_as)
    else: 
        print(f'{file_save_as} is existing.')



../data/parquet/en.train-00028-of-00041.parquet is existing.
../data/parquet/fr.train-00011-of-00017.parquet is existing.


In [92]:
parquet_file_paths

[(['/datasets/wikimedia/wikipedia/resolve/main/20231101.en/train-00028-of-00041.parquet?download=true',
   '188'],
  'en'),
 (['/datasets/wikimedia/wikipedia/resolve/main/20231101.fr/train-00011-of-00017.parquet?download=true',
   '169'],
  'fr')]

In [98]:
import pandas as pd
df = pd.read_parquet(file_save_as, engine='fastparquet')
text = " ".join(df["text"].values.tolist())

print(text[:100])


L' aérodrome du nord-est de la Floride , anciennement aérodrome Saint-Augustine, est situé à  au nor


323

In [100]:
len(text)

293876075